In [1]:
reset()


In [7]:
R.<x> = PolynomialRing(ZZ)  # Define a polynomial ring over the rationals
d=[]
with open('filtered_list.txt', 'r') as file:
    content = file.read()

# Strip out 'L = ' to get the actual list part
content = content.strip().split('=', 1)[-1].strip()

# Replace ^ with ** for exponentiation
content = content.replace('^', '**')

# Use eval to evaluate the list of polynomials
d = eval(content)

In [16]:
def C(L,g): # L=[A,B,C] : y^2=x^3+Ax^2+Bx+C
    A,B,C=L[0],L[1],L[2]
    
    def sf_divisors(N):
        if is_prime(abs(N)):
            return [1,N]
        L = []
        div = divisors(abs(N))
        for x in div:
            if is_squarefree(x):
                L.append(x)
        return L
    
    f = x^3 + A*x^2 + B*x + C
    var('u,v')
    f1 = u^2 - f
    g1 = v^2 - g
    R0 = f1.resultant(g1,x).subs(u=0,v=0)
    #print("Resultant:", R0)
    SF_d = sf_divisors(int(R0))
    #print(SF_d)
    sols = []

    for d in SF_d:
        d = int(d)
        E = EllipticCurve([0,A*int(d),0,B*int(d)^2,C*int(d)^3])
        L = E.integral_points()
        L1 = []
        for P in L: 
            if P[1]!=0: # if y!=0
                L1.append(P)
        for Q in L1: # Q are integer points on E with y!=0
            if int(Q[0])%d==0:
                sols.append(Q[0]/d)        
    for d1 in SF_d:
        d = -d1
        E = EllipticCurve([0,A*int(d),0,B*int(d)^2,C*int(d)^3])
        L = E.integral_points()
        L1 = []
        for P in L:
            if P[1]!=0:
                L1.append(P)
        for Q in L1:
            if int(Q[0])%d==0:
                sols.append(Q[0]/d)
                
    H =  f * g
    SOLS = []
    for z in sols:
        S = H.subs(x=z)
        if is_square(int(S)) and S!=0:
            SOLS.append([z,sqrt(S)])
    return SOLS,f*g


############ code for finding the points modp, p prime ###################

def poly_eval(coeffs, x, p):
    """
    Evaluate polynomial F(x) mod p, 
    where coeffs[i] is the coefficient of x^i.
    """
    val = 0
    power_of_x = 1
    for c in coeffs:
        val = (val + c * power_of_x) % p
        power_of_x = (power_of_x * x) % p
    return val

def poly_deriv(coeffs):
    """
    Given F(x) with coeffs[i] = coefficient of x^i,
    return coeffs of F'(x), also in ascending order.
    F'(x) = a1 + 2*a2*x + 3*a3*x^2 + ... + n*an*x^(n-1).
    """
    deriv = []
    for i in range(1, len(coeffs)):
        deriv.append(i * coeffs[i])  # integer derivative
    return deriv

def poly_eval_deriv(deriv_coeffs, x, p):
    """
    Evaluate F'(x) mod p, given derivative coefficients in ascending order.
    """
    val = 0
    power_of_x = 1
    for c in deriv_coeffs:
        val = (val + c * power_of_x) % p
        power_of_x = (power_of_x * x) % p
    return val

def find_solutions_and_check_singularity(coeffs, p):
    """
    Given a polynomial F(x) (coeffs in ascending order),
    and prime p:
      1) Find all (x, y) mod p satisfying y^2 = F(x).
      2) Check if the curve y^2 - F(x) is singular at any solution.
         G(x,y)=0 with dG/dx=0 and dG/dy=0 => bad reduction.
      3) Return (list_of_solutions, is_good_reduction).
    """
    solutions = []
    found_singular = False
    
    # Derivative F'(x):
    deriv_coeffs = poly_deriv(coeffs)
    
    # Precompute squares mod p => squares_map[val] = list of y s.t. y^2=val
    squares_map = {}
    for y in range(p):
        sq = (y * y) % p
        squares_map.setdefault(sq, []).append(y)
    
    # Loop x in [0..p-1]
    for x_val in range(p):
        fx = poly_eval(coeffs, x_val, p)  # F(x_val) mod p
        if fx in squares_map:
            # We have solutions y^2 = fx
            for y_val in squares_map[fx]:
                solutions.append((x_val, y_val))
                
                # Check partial derivatives for singularity:
                # dG/dx(x_val,y_val) = -F'(x_val), dG/dy = 2*y_val.
                dGdx = (- poly_eval_deriv(deriv_coeffs, x_val, p)) % p
                
                if dGdx == 0:
                    # => the partial wrt x is 0 mod p
                    if p != 2:
                        # Then we also need 2*y_val == 0 => y_val==0 mod p (since 2 is invertible).
                        if y_val == 0:
                            found_singular = True
                    else:
                        # If p=2 => 2==0 mod 2 => partial wrt y is identically 0 => automatically singular if dGdx=0 too.
                        found_singular = True
    
    is_good = not found_singular
    return solutions, is_good
    
def mod5mod7(F):
    A=[]
    F_coeffs = F.coefficients()
    primes_to_test = [5, 7]
    for p in primes_to_test:
        sols, is_good = find_solutions_and_check_singularity(F_coeffs, p)
        A.append(len(sols))
    return A

The following code computes the integer points of $y^2=f(x)$ where $f(x)$ belongs to the list : ${\tt{filtered\_list.txt}}$, which contains all the genus 2
curves with discriminant $\le 10^6$ 
and 2-Selmer group $=1.$ So $r={\rm{rank(J}}({\mathbb{Q}}))\le 1.$ Using the improved Coleman bound due to Stoll, we get
$$|C({\mathbb{Q}})|\le |{\tilde{C}}({\bf F}_p)| + 2r $$
where $p>2g(C)$ is a prime of good reduction for the curve $C.$ The last column has the number of points $\mod{5}$ and $\mod{7}$ without the possible points at infinity. We have to add them for instance if $f(x)$ is monic.

In [17]:
R.<x> = QQ[] 
for i in range(len(d)):
    f=d[i][0]
    a,b,c=f.coefficient(2),f.coefficient(1),f.coefficient(0)
    #print(i)
    try:
        A=C( [a,b,c], d[i][1] )
        print(i,":",C( [a,b,c], d[i][1] ),",",mod5mod7(R(A[1])))
    except RuntimeError as e:
        print(i,"","runtime-error")

0 : ([], x^6 + 12*x^5 + 18*x^4 - 18*x^3 - 19*x^2 + 6*x + 5) , [5, 5]
1 : ([], x^6 + 14*x^5 + 25*x^4 - 58*x^3 + 18*x^2 + 8*x - 3) , [1, 7]
2 : ([[2, 30]], 5*x^6 - 36*x^5 + 18*x^4 + 92*x^3 + 133*x^2 + 76*x + 24) , [3, 7]
3 : ([[0, 8]], -3*x^6 + 2*x^5 + 27*x^4 - 10*x^3 - 71*x^2 + 20*x + 64) , [9, 8]
4 : ([], 5*x^6 + 98*x^5 + 651*x^4 + 1588*x^3 + 655*x^2 + 98*x + 5) , [3, 4]
5 : ([], 4*x^5 - 64*x^4 - 28*x^3 + 116*x^2 + 112*x + 28) , [3, 4]
6 : ([], 5*x^6 - 12*x^5 + 16*x^4 - 24*x^2 + 32*x - 20) , [6, 6]
7 : ([], x^6 + 2*x^5 - 19*x^4 - 30*x^3 + 114*x^2 + 112*x - 207) , [3, 9]
8 : ([], -3*x^6 - 8*x^5 + 2*x^4 + 34*x^3 + 41*x^2 + 6*x - 15) , [3, 7]
9 : ([[-1, 1]], x^6 + 10*x^4 + 10*x^3 + 21*x^2 + 42*x + 21) , [9, 3]
10 : ([[-2, 3]], 5*x^6 + 6*x^5 - 19*x^4 - 10*x^3 + 26*x^2 - 4*x - 7) , [2, 10]
11 : ([], -3*x^6 - 4*x^5 + 32*x^4 + 26*x^3 - 116*x^2 - 36*x + 133) , [3, 9]
12 : ([[0, 1]], 5*x^6 + 24*x^5 - 2*x^4 - 30*x^3 + x^2 + 6*x + 1) , [4, 5]
13 : ([], x^6 + 2*x^5 - 11*x^4 - 30*x^3 - 26*x^2 - 32*